In [1]:
import sys
sys.path.append("D:/ds/code/")
sys.path.append("D:/ds/code/implementation")

from implementation.multiind.indicators import *

In [2]:
import sys
import logging

from configparser import NoOptionError

from pymongo import MongoClient
import twieds

config = twieds.setup("../implementation/logs/mi_alloc.log", "../implementation/settings/locinf.ini")

# connect to the MongoDB
logging.info("Connecting to MongoDB...")
client = MongoClient(config.get("mongo", "address"), config.getint("mongo", "port"))
logging.info("Connected to MongoDB.")

# select the database and collection based off config
try:
    db = client[config.get("mongo", "database")]
    col = db["geotweets"]  # config.get("mongo", "collection")]
except NoOptionError:
    logging.critical("Cannot connect to MongoDB database and collection. Config incorrect?")
    sys.exit()

INFO:root:-----------
2016-02-21 23:05:06,869 [MainThread ] INFO    - -----------
INFO:root:Initialised logging for: C:\Users\Humphrey\Anaconda3\lib\site-packages\ipykernel\__main__.py 12044
2016-02-21 23:05:06,870 [MainThread ] INFO    - Initialised logging for: C:\Users\Humphrey\Anaconda3\lib\site-packages\ipykernel\__main__.py 12044
INFO:root:-----------
2016-02-21 23:05:06,871 [MainThread ] INFO    - -----------
INFO:root:Connecting to MongoDB...
2016-02-21 23:05:06,873 [MainThread ] INFO    - Connecting to MongoDB...
INFO:root:Connected to MongoDB.
2016-02-21 23:05:06,876 [MainThread ] INFO    - Connected to MongoDB.


In [3]:
from Polygon import Polygon
from Polygon import Error as PolygonError
from geopy.distance import vincenty

def get_centre(polys):
    p = Polygon()
    for i in polys:
        p.addContour(i[0])

    try:
        return p.center()
    except PolygonError:
        return None
    
def get_distance(centre, real):
    return vincenty((centre[1], centre[0]), (real[1], real[0])).km

In [4]:
cursor = col.find({'geo': {'$ne': None}})

geonames = LocFieldIndicator(config)
gisgraphy = GISLocFieldIndicator(config)

geoav = 0
gisav = 0
count = 0

allocid = 0
for doc in cursor:
    print(doc['user']['location'])
    geo = geonames.get_loc(doc['user']['location'])
    gis = gisgraphy.get_loc(doc['user']['location'])
    print(len(geo), len(gis))
    
    point = (doc["geo"]["coordinates"][1], doc["geo"]["coordinates"][0])
    geoav += get_distance(get_centre(geo), point)
    gisav += get_distance(get_centre(gis), point)
    count += 1
    if count > 100:
        break

INFO:urllib3.connectionpool:Starting new HTTP connection (1): api.geonames.org
2016-02-21 23:05:09,166 [MainThread ] INFO    - Starting new HTTP connection (1): api.geonames.org
DEBUG:urllib3.connectionpool:"GET /search?q=Greenbelt+MD&orderBy=relevance&style=FULL&username=humph&type=json&fuzzy=0.8&maxRows=5 HTTP/1.1" 200 None
INFO:urllib3.connectionpool:Starting new HTTP connection (1): spotlight.sztaki.hu
2016-02-21 23:05:09,430 [MainThread ] INFO    - Starting new HTTP connection (1): spotlight.sztaki.hu
DEBUG:urllib3.connectionpool:"GET /rest/annotate?support=20&text=Greenbelt+MD&confidence=0.2 HTTP/1.1" 200 None
INFO:urllib3.connectionpool:Starting new HTTP connection (1): dbpedia.org
2016-02-21 23:05:09,612 [MainThread ] INFO    - Starting new HTTP connection (1): dbpedia.org
DEBUG:urllib3.connectionpool:"GET /data/Greenbelt,_Maryland.json HTTP/1.1" 200 37143
DEBUG:urllib3.connectionpool:"GET /data/Maryland.json HTTP/1.1" 200 1091393
DEBUG:root:   Message =  2 resources [.#]
DEBUG

Greenbelt MD
 :

DEBUG:urllib3.connectionpool:"GET /fulltext?format=json&q=Greenbelt&suggest=true HTTP/1.1" 200 None
DEBUG:root: Greenbelt =  2 geonames [..]
DEBUG:urllib3.connectionpool:"GET /fulltext?format=json&q=MD&suggest=true HTTP/1.1" 200 None
DEBUG:root:        MD =  10 geonames [..=..]


 Greenbelt
 : MD
152 7
San Antonio, TX

DEBUG:urllib3.connectionpool:"GET /search?q=San+Antonio%2C+TX&orderBy=relevance&style=FULL&username=humph&type=json&fuzzy=0.8&maxRows=5 HTTP/1.1" 200 None
DEBUG:root:  LocField =  1 geonames [.]
DEBUG:urllib3.connectionpool:"GET /fulltext?format=json&q=San+Antonio%2C+TX&suggest=true HTTP/1.1" 200 None
DEBUG:root:San Antonio, TX =  10 geonames [.....]


...

 Greenbelt
 : MD
152

DEBUG:urllib3.connectionpool:"GET /search?q=Greenbelt+MD&orderBy=relevance&style=FULL&username=humph&type=json&fuzzy=0.8&maxRows=5 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:"GET /rest/annotate?support=20&text=Greenbelt+MD&confidence=0.2 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:"GET /data/Greenbelt,_Maryland.json HTTP/1.1" 200 37143
DEBUG:urllib3.connectionpool:"GET /data/Maryland.json HTTP/1.1" 200 1091393
DEBUG:root:   Message =  2 resources [.#]
DEBUG:root:  LocField =  0 geonames []
DEBUG:urllib3.connectionpool:"GET /fulltext?format=json&q=Greenbelt+MD&suggest=true HTTP/1.1" 200 None
DEBUG:root:Greenbelt MD =  0 geonames []


 7
Greenbelt MD
 :

DEBUG:urllib3.connectionpool:"GET /fulltext?format=json&q=Greenbelt&suggest=true HTTP/1.1" 200 None
DEBUG:root: Greenbelt =  2 geonames [..]
DEBUG:urllib3.connectionpool:"GET /fulltext?format=json&q=MD&suggest=true HTTP/1.1" 200 None
DEBUG:root:        MD =  10 geonames [..=..]


 Greenbelt
 : MD
152 7


In [ ]:
print (geoav / count, gisav / count)